### 🤖 **Interactive Multi-LLM Interface with Gradio**

This project implements a unified web interface that allows users to interact with three of the most powerful Large Language Models (LLMs) on the market: GPT-40-mini (OpenAI), Claude 3 Haiku (Anthropic), and Gemini 2.5 Flash (Google).

The application uses the Gradio library to create the visual frontend and leverages the streaming capabilities of each API, allowing responses to be generated text-to-text in real time, thus improving the user experience. The code is designed in a modular fashion, with specific functions to handle the connection with each AI provider independently.

In [129]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import anthropic as ant
import google.generativeai as genai
from IPython.display import Markdown, display

##### API Key Management and Verification

In [130]:
load_dotenv(override=True)

API_KEYS = {
    "OpenAI": ("OPENAI_API_KEY", 8),
    "Anthropic": ("ANTHROPIC_API_KEY", 7),
    "Google": ("GOOGLE_API_KEY", 2)
}

keys = {label: os.getenv(env) for label, (env, _) in API_KEYS.items()}

def check_key(label, prefix_len):
    key = keys[label]
    if key:
        print(f"{label} API Key exists and begins {key[:prefix_len]}")
    else:
        print(f"{label} API Key not set (optional)")

for label, (env_name, prefix_len) in API_KEYS.items():
    check_key(label, prefix_len)

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI


##### Client and Model Configuration

In [131]:
openai  = OpenAI(api_key=keys["OpenAI"])
anthropic = ant.Anthropic(api_key=keys["Anthropic"])
genai.configure(api_key=keys["Google"])

In [150]:
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = "models/gemini-2.5-flash"
max_tokens = 5120
system_prompt = "You are a helpful assistant."

##### Generation Function for OpenAI (GPT)

In [101]:
def call_gpt(prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    stream = openai.chat.completions.create(
        model=gpt_model,
        messages=messages,
        stream=True
    )
    result = ""
    #display_handle = display(Markdown("⏳ _Generando..._"), display_id=True)
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        #display_handle.update(Markdown(result))
        yield result

In [102]:
call_gpt("What is the date today?")

<generator object call_gpt at 0x0000020B7BBCC640>

##### Generation Function for Anthropic (Claude)

In [ ]:
def call_claude(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
    response = anthropic.messages.stream(
        model=claude_model,
        system=system_prompt,
        messages=messages,
        max_tokens=max_tokens
    )
    accumulated_text = ""
    #display_handle = display(Markdown("⏳ Generando respuesta..."), display_id=True)
    with response as stream:
        for text in stream.text_stream:
            accumulated_text += text
            #display_handle.update(Markdown(accumulated_text))
            yield accumulated_text

In [ ]:
call_claude("Explain the Transformer architecture to an aspiring AI engineer")

##### Generation function for Google (Gemini)

In [175]:
def call_gemini(prompt):
    config = genai.types.GenerationConfig(
        max_output_tokens=max_tokens,
        temperature=0.7
    )

    model = genai.GenerativeModel(
        model_name=gemini_model,
        system_instruction=system_prompt
    )

    response = model.generate_content(prompt, stream=True, generation_config=config)
    accumulated_text = ""
    #display_handle = display(Markdown("⏳ Generando respuesta..."), display_id=True)
    for chunk in response:
        if chunk.text:
            accumulated_text += chunk.text
            #display_handle.update(Markdown(accumulated_text))
            yield accumulated_text

In [176]:
call_gemini("Explain the Transformer architecture to an aspiring AI engineer")

<generator object call_gemini at 0x0000020B7BBCDA40>

### Model Selection Logic (Dispatcher)

In [191]:
def stream_model(prompt, model_name):
    models = {
        "GPT": call_gpt,
        "Claude": call_claude,
        "Gemini": call_gemini
    }

    if model_name not in models:
        raise ValueError(f"Unknown model '{model_name}'. Available: {list(models.keys())}")

    yield from models[model_name](prompt)


In [185]:
stream_model("Dime un chiste", "Ollama")

<generator object stream_model at 0x0000020B7C36EF80>

### Building and Testing the Interface with Gradio

In [19]:
def shout(text):
    return text.upper()

In [ ]:
shout("Hello World")

In [ ]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

##### Gradio dark mode

In [ ]:
force_dark_mode = """
(function() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
})();
"""

demo = gr.Interface(
    fn=shout, 
    inputs="textbox", 
    outputs="textbox", 
    flagging_mode="never"
)

demo.launch(share=True, js=force_dark_mode)

### Application Launch

In [192]:
message_input = gr.Textbox(label="Your message:", lines=7)
model_selector = gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model", value="GPT")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_model,
    title="LLMs", 
    inputs=[message_input, model_selector], 
    outputs=[message_output], 
    examples=[
       ["Explain the Transformer architecture to a layperson", "GPT"],
       ["Explain the Transformer architecture to an aspiring AI engineer", "Claude"],
       ["Explain the Transformer architecture to an Software engineer", "Gemini"]
    ],
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7900
* To create a public link, set `share=True` in `launch()`.
